<a href="https://colab.research.google.com/github/KirtiNayak11/ML/blob/main/FaceEmotionRecognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [191]:
import os

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from glob import glob
import seaborn as sns
from PIL import Image
np.random.seed(11) # It's my lucky number
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV
from sklearn.metrics import accuracy_score
import itertools

import keras
from keras.utils.np_utils import to_categorical # used for converting labels to one-hot-encoding
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras import backend as K
#from keras.layers.normalization import BatchNormalization
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.optimizers import Adam, RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
from keras.wrappers.scikit_learn import KerasClassifier
#from keras.applications.resnet50 import ResNet50
from keras import backend as K
from tensorflow import keras
from tensorflow.keras import layers, models

#standardizing the images

In [192]:
picture_size=48

In [193]:
import cv2

In [194]:
def load_and_preprocess_data(data_folder):
    data = []
    labels = []

    for emotion in os.listdir(data_folder):
        emotion_folder = os.path.join(data_folder, emotion)
        for image_filename in os.listdir(emotion_folder):
            image_path = os.path.join(emotion_folder, image_filename)
            image = cv2.imread(image_path)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
            image = cv2.resize(image, (48, 48))  # Resize to a common size
            image = image / 255.0  # Normalize pixel values to [0, 1]
            data.append(image)
            labels.append(emotion)

    return np.array(data), np.array(labels)


In [195]:
data_folder='/content/drive/MyDrive/DL_practice/Project/FaceEmotionDetection/train'
xtrain, ytrain = load_and_preprocess_data(data_folder)

In [196]:
ytrain_size=ytrain

In [197]:
num_classes=len(set(ytrain_size))
num_classes

7

In [198]:
# Define the path to your test data folder
test_data_folder = '/content/drive/MyDrive/DL_practice/Project/FaceEmotionDetection/test'

# Load and preprocess the test data
xtest,ytest = load_and_preprocess_data(test_data_folder)

In [199]:
xtrain.shape

(51, 48, 48)

In [200]:
ytrain.shape

(51,)

In [201]:
xtest.shape

(45, 48, 48)

In [203]:
from keras.utils import to_categorical

# Initialize the label encoder
label_encoder = LabelEncoder()

# Encode the string labels into integers for both training and testing
ytrain = label_encoder.fit_transform(ytrain)
ytest = label_encoder.transform(ytest)

# Convert integer-encoded labels into one-hot encoded format
#ytrain = to_categorical(ytrain, num_classes)
#ytest = to_categorical(ytest, num_classes)

In [204]:
ytrain

array([4, 4, 4, 4, 4, 4, 4, 3, 3, 3, 3, 3, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 1, 1, 1, 6, 6, 6, 6, 6, 6, 6, 5, 5, 5, 5, 5, 5, 5,
       2, 2, 2, 2, 2, 2, 2])

In [206]:
ytest

array([1, 1, 1, 1, 1, 1, 3, 3, 3, 3, 3, 3, 3, 5, 5, 5, 5, 5, 5, 2, 2, 2,
       2, 2, 2, 6, 6, 6, 6, 6, 6, 6, 0, 0, 0, 0, 0, 0, 0, 4, 4, 4, 4, 4,
       4])

In [207]:
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(7, activation='softmax')  # 7 emotion classes
])

In [208]:
model.compile(optimizer='SGD',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


In [209]:
xtrain.shape

(51, 48, 48)

In [210]:
reshaped_xtrain = xtrain.reshape((51, 48, 48, 1))
reshaped_xtrain

array([[[[0.01568627],
         [0.        ],
         [0.        ],
         ...,
         [0.10588235],
         [0.09411765],
         [0.09803922]],

        [[0.00392157],
         [0.        ],
         [0.        ],
         ...,
         [0.10196078],
         [0.09019608],
         [0.10196078]],

        [[0.        ],
         [0.        ],
         [0.        ],
         ...,
         [0.10588235],
         [0.10196078],
         [0.10196078]],

        ...,

        [[0.        ],
         [0.        ],
         [0.        ],
         ...,
         [0.13333333],
         [0.1254902 ],
         [0.11764706]],

        [[0.        ],
         [0.        ],
         [0.        ],
         ...,
         [0.12941176],
         [0.12156863],
         [0.11764706]],

        [[0.        ],
         [0.        ],
         [0.        ],
         ...,
         [0.11764706],
         [0.11372549],
         [0.11764706]]],


       [[[0.15294118],
         [0.29411765],
         [0.30

In [211]:
ytrain.shape

(51,)

In [212]:
batch_size = 32

# Train the model with batch processing
model.fit(reshaped_xtrain, ytrain, epochs=1)

2/2 [==============================] - 1s 60ms/step - loss: 1.9451 - accuracy: 0.1765


The code you've shared seems to be mostly correct for loading test data from the specified folder and then using your model to make predictions on that test data. However, there are a couple of important points to consider:

Data Preprocessing: Before making predictions, you should preprocess the test data in the same way you preprocessed the training data. This includes resizing the images, converting them to grayscale, normalizing pixel values, and ensuring that they have the same shape as the input data that your model expects.

Model Input Shape: Ensure that the test data you load has the same shape as what your model expects as input. Based on your model architecture mentioned earlier, it appears to expect images of size (48, 48, 1), which are grayscale images of 48x48 pixels.

In [213]:
# Ensure the input shape matches what your model expects
# (Assuming model is already defined and compiled)
ypred = model.predict(xtest)
ypred

2/2 [==============================] - 0s 24ms/step


array([[0.14912494, 0.14530914, 0.14657196, 0.15048496, 0.13189963,
        0.12849556, 0.14811398],
       [0.1559071 , 0.14324355, 0.14167073, 0.14746901, 0.13568181,
        0.12872869, 0.14729911],
       [0.15308712, 0.14686008, 0.140874  , 0.1449103 , 0.13674296,
        0.13236283, 0.14516276],
       [0.14712402, 0.14626546, 0.1443683 , 0.14691578, 0.14125083,
        0.13136223, 0.14271335],
       [0.14646253, 0.14469606, 0.14284712, 0.14850251, 0.13494144,
        0.13139836, 0.15115193],
       [0.15524916, 0.14218695, 0.14393033, 0.14915976, 0.13467292,
        0.12486364, 0.14993723],
       [0.15126994, 0.14516008, 0.1462022 , 0.14844853, 0.13168445,
        0.12668258, 0.15055212],
       [0.15112203, 0.14743355, 0.14180318, 0.15091302, 0.13614574,
        0.12599577, 0.14658667],
       [0.15491106, 0.15045506, 0.14436847, 0.14585719, 0.13312343,
        0.12114652, 0.15013838],
       [0.14785999, 0.14339446, 0.14372838, 0.1514152 , 0.13641125,
        0.13188551, 0.1

In [216]:
from sklearn.metrics import confusion_matrix , classification_report
y_pred_classes = [np.argmax(element) for element in ypred]
y_pred_classes[:5]
print("Classification Report: \n", classification_report(ytest, y_pred_classes))

Classification Report: 
               precision    recall  f1-score   support

           0       0.14      0.57      0.23         7
           1       0.00      0.00      0.00         6
           2       0.00      0.00      0.00         6
           3       0.14      0.14      0.14         7
           4       0.00      0.00      0.00         6
           5       0.00      0.00      0.00         6
           6       0.40      0.57      0.47         7

    accuracy                           0.20        45
   macro avg       0.10      0.18      0.12        45
weighted avg       0.11      0.20      0.13        45



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [215]:
ytest[:5]

array([1, 1, 1, 1, 1])